In [2]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=afbd13a6229b3f8ac983122ca687d11ed8692da625c60ddc1844173293bd7994
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 34.4 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import torch
import random
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle
import json

In [5]:
def data_prep(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            try:
                item = json.loads(line)
                data.append(item)
            except json.JSONDecodeError:
                print(f"Ignoring line: {line}")

    df = pd.DataFrame(data)
    df['id'] = [random.randint(1, 1000000) for _ in range(len(df))]
    return df

df = data_prep(file_path=r'/content/drive/MyDrive/News_Category_Dataset_v3.json')

In [6]:
df.isnull().sum()

link                 0
headline             0
category             0
short_description    0
authors              0
date                 0
id                   0
dtype: int64

In [7]:
df_2 = df.drop(['link', 'category', 'authors'], axis=1)

In [8]:
df_2.head()

,headline,short_description,date,id
0,Over 4 Million Americans Roll Up Sleeves For O...,Health experts said it is too early to predict...,2022-09-23,258557
1,"American Airlines Flyer Charged, Banned For Li...",He was subdued by passengers and crew when he ...,2022-09-23,613841
2,23 Of The Funniest Tweets About Cats And Dogs ...,"""Until you have a dog you don't understand wha...",2022-09-23,184164
3,The Funniest Tweets From Parents This Week (Se...,"""Accidentally put grown-up toothpaste on my to...",2022-09-23,579141
4,Woman Who Called Cops On Black Bird-Watcher Lo...,Amy Cooper accused investment firm Franklin Te...,2022-09-22,798343


In [10]:
df_2.rename(columns = {'headline' : 'original_title'}, inplace = True)
df_2.rename(columns = {'short_description' : 'abstract'}, inplace = True)
df_2.rename(columns = {'date' : 'year'}, inplace = True)

In [11]:
df_2.head()

,original_title,abstract,year,id
0,Over 4 Million Americans Roll Up Sleeves For O...,Health experts said it is too early to predict...,2022-09-23,258557
1,"American Airlines Flyer Charged, Banned For Li...",He was subdued by passengers and crew when he ...,2022-09-23,613841
2,23 Of The Funniest Tweets About Cats And Dogs ...,"""Until you have a dog you don't understand wha...",2022-09-23,184164
3,The Funniest Tweets From Parents This Week (Se...,"""Accidentally put grown-up toothpaste on my to...",2022-09-23,579141
4,Woman Who Called Cops On Black Bird-Watcher Lo...,Amy Cooper accused investment firm Franklin Te...,2022-09-22,798343


In [12]:
#Embedding instance
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

In [14]:
#Vector conversion
embeddings = model.encode(df_2.abstract.to_list(), show_progress_bar=True)

Batches:   0%|          | 0/6548 [00:00<?, ?it/s]

In [15]:
embeddings.shape

(209527, 768)

In [16]:
#Indexing documents with Faiss
embeddings = np.array([i for i in embeddings]).astype('float32')

index = faiss.IndexFlatL2(embeddings.shape[1])
index = faiss.IndexIDMap(index)
index.add_with_ids(embeddings, df_2.id.values)
with open("faiss_index.pickle", "wb") as file:
    index_data = faiss.serialize_index(index)
    pickle.dump(index_data, file)

In [17]:
D, I = index.search(np.array([embeddings[5415]]), k=10)
print(f'L2 distance: {D.flatten().tolist()}\n\news IDs: {I.flatten().tolist()}')

L2 distance: [0.0, 154.99685668945312, 159.84678649902344, 159.84678649902344, 166.29608154296875, 168.82891845703125, 168.83428955078125, 169.21974182128906, 170.1654510498047, 170.32891845703125]

ews IDs: [823010, 485271, 353722, 445473, 748378, 592678, 97307, 487447, 571858, 716555]


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
def id2details(df_2, I, column):
    return [list(df_2[df_2.id == idx][column]) for idx in I[0]]

In [20]:
id2details(df_2=df_2, I=I, column = 'original_title')

[['Boeing Expects 737 MAX Planes To Return To Service Mid-2020'],
 ['15 Money-Saving Tricks From Real Brides',
  'Addyson Gale Cessna, 13 Pound 12 Ounce Baby Girl, Born In Pennsylvania (VIDEO)'],
 ['VietJetAir, Vietnamese Airline, Fined For In-Flight Bikini Show (VIDEO)'],
 ["'One Nation Under Stress,' With To-Do Lists And Yoga For All",
  'VietJetAir, Vietnamese Airline, Fined For In-Flight Bikini Show (VIDEO)'],
 ["Gwen Stefani And Blake Shelton Aren't Shy With The PDA At Pre-Grammys Bash",
  "Success Isn't for You"],
 ["Detroit Station: Man Who Blamed Mom's Death On Trump Lied About Date She Died (UPDATE)"],
 ['Movie Review: Child 44 -- Leave This One Behind'],
 ["5 Things I Didn't Know About My Mom Until I Had My Daughter"],
 ["Healthy Snacks For Kids? Blended Fruit Pouches Aren't 'As Perfect As They Seem'",
  "Fearless 'Push Girl' Auti Angel: 'Once You Fall In Love With The Person, You Forget About The Wheelchair'"],
 ['Target Follow Up: Speaking to the Manager']]

In [21]:
user_query = 'Trump'

In [22]:
def vector_search(query, model, index, num_results=10):
    vector = model.encode(list(query))
    D, I = index.search(np.array(vector).astype("float32"), k=num_results)
    return D, I

D, I = vector_search([user_query], model, index, num_results=10)



In [24]:
id2details(df_2=df_2, I = I, column = 'original_title')

[['ACLU Giving Away Free Pocket Constitutions Until Election Day'],
 ["Vladimir Putin's Bare-Chested Vacation Snap Becomes Summer's Best Meme"],
 ['Why You Should Stop Feeding Your Baby From Trendy Little Food Pouches',
  'This State Just Dug Deep Into Voting Irregularities. It Found Nothing Close To Widespread Voter Fraud.'],
 ["Seth Meyers Hilariously Imagines The Trump Family's First Thanksgiving"],
 ["'Earth On The Edge': EU Agency Confirms 2016 As Hottest Year On Record"],
 ["Ivanka Trump's Weird Scientist Snap Sparks A Huge 'Photoshop Battle'"],
 ['Why Republican Voters Decided On Trump',
  'Watch Mr. Munro, An Australian Penguin, Paint And Craft (VIDEO)'],
 ["House Votes For More Punishments For Undocumented Immigrants, 'Sanctuary Cities'"],
 ['Republicans And Democrats Have Very Different Ideas About What Saved A Congressional Ethics Watchdog'],
 ["LeBron James Says Locker Room Talk Isn't What 'That Guy' Thinks It Is"]]